In [12]:
import cv2
import numpy as np

def main():

    i = 0      # カウント変数
    th = 30  # 差分画像の閾値
    
    #保存
    fmt = cv2.VideoWriter_fourcc(*'MJPG')
    writer = cv2.VideoWriter('outtest2.avi', fmt, 30.0, (640, 480))

    # カメラのキャプチャ
    #cap = cv2.VideoCapture("output_02.avi")
    cap = cv2.VideoCapture(0)
    # 最初のフレームを背景画像に設定
    ret, bg = cap.read()
    #print(bg)
    # グレースケール変換
    #bg = cv2.cvtColor(bg, cv2.COLOR_BGR2RGB)
    #bg = cv2.cvtColor(cap.read()[1], cv2.COLOR_RGB2GRAY)
    #ret,bg=cap.read()
    while(cap.isOpened()):
        # フレームの取得
        ret,frame = cap.read()
        cv2.imshow("Frame",frame)
        # グレースケール変換
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #ray = cv2.cvtColor(cap.read()[1], cv2.COLOR_RGB2GRAY)
        #print(gray)
        # 差分の絶対値を計算
        
        b = cv2.absdiff(frame[:,:,0],bg[:,:,0])
        g = cv2.absdiff(frame[:,:,1],bg[:,:,1])
        r = cv2.absdiff(frame[:,:,2],bg[:,:,2])
        #mask = cv2.absdiff(gray, bg)
        # 差分画像を二値化してマスク画像を算出
        #mask[mask < th] = 0
        #mask[mask >= th] = 255
        g[g < th] = 0
        g[g >= th] = 1
        r[r < th] = 0
        r[r >= th] = 1
        b[b < th] = 0
        b[b >= th] = 1
        mask=g+r+b
        mask[mask == 0]=0
        mask[mask != 0]=255
        frame[mask==0]=0
        # フレームとマスク画像を表示
        
        if ret==True:
            #frame = cv2.flip(frame,0)

            # write the flipped frame
            mask=cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
            writer.write(mask)
            cv2.imshow("Mask", mask)
            cv2.imshow("Result", frame)
        
        #i += 1    # カウントを1増やす
        
        # 背景画像の更新（一定間隔）
        #if(i > 30):
        #    ret, bg = cap.read()
            #bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY) 
        #    i = 0 # カウント変数の初期化
            
        # qキーが押されたら途中終了
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()

In [2]:
bg

NameError: name 'bg' is not defined

IndexError: list index out of range